In [1]:
import pandas as pd
products = pd.read_csv('data_skinrec_bismillah.csv')
products.head()

,product_href,product_name,product_type,brand,notable_effects,skintype,price,description,picture_src,sensitive,...,refreshing,hydrating,balancing,pore_care,no_whitecast,oil_control,black_spot,brightening,soothing,anti_aging
0,https://www.beautyhaul.com/product/detail/bubb...,ACWELL Bubble Free PH Balancing Cleanser,Face Wash,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,209000,Mengangkat kotoran dan menghapus makeup dalam ...,https://www.beautyhaul.com/assets/uploads/prod...,0,...,0,0,0,0,0,0,0,1,0,0
1,https://www.sociolla.com/face-wash/62452-ph-ba...,ACWELL pH Balancing Soothing Cleansing Foam,Face Wash,ACWELL,"Soothing, Balancing","Normal, Dry, Combination",181800,Membersihkan dan menenangkan kulit sensitif de...,https://images.soco.id/8f08ced0-344d-41f4-a15e...,0,...,0,0,1,0,0,0,0,0,1,0
2,https://www.sociolla.com/toner/15871-licorice-...,Acwell Licorice pH Balancing Cleansing Toner,Toner,ACWELL,"Soothing, Balancing","Normal, Dry, Oily, Combination, Sensitive",149000,"Mengangkat sisa kotoran, debu, dan make up sek...","https://www.soco.id/cdn-cgi/image/w=73,format=...",1,...,0,0,1,0,0,0,0,0,1,0
3,https://www.beautyhaul.com/product/detail/aqua...,ACWELL Aquaseal Soothing Tonic,Toner,ACWELL,"Acne-Free, Pore-Care, Brightening, Anti-Aging",Oily,290000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,0,...,0,0,0,0,0,0,0,1,0,0
4,https://www.sociolla.com/essence/38023-licoric...,Licorice pH Balancing Essence Mist,Toner,ACWELL,"Brightening, Soothing","Normal, Dry",194650,Essens mist dengan kelembapan tinggi yang memb...,"https://www.sociolla.com/cdn-cgi/image/w=425,f...",0,...,0,0,0,0,0,0,0,1,1,0


In [2]:
print(products.columns)

Index(['product_href', 'product_name', 'product_type', 'brand',
       'notable_effects', 'skintype', 'price', 'description', 'picture_src',
       'sensitive', 'combination', 'oily', 'dry', 'normal', 'rating',
       'keluhan', 'kulit_kusam', 'jerawat', 'bekas_jerawat', 'pori_pori_besar',
       'flek_hitam', 'garis_halus_dan_kerutan', 'komedo',
       'warna_kulit_tidak_merata', 'kemerahan', 'kulit_kendur', 'moisturizing',
       'skin_barrier', 'uv_protection', 'acne_free', 'refreshing', 'hydrating',
       'balancing', 'pore_care', 'no_whitecast', 'oil_control', 'black_spot',
       'brightening', 'soothing', 'anti_aging'],
      dtype='object')


In [7]:
def recommend_skincare(product_type, skin_types, skin_problems, notable_effects):
    # Filter produk berdasarkan product_type
    filtered_df = products[products['product_type'] == product_type]
    
    # Filter produk berdasarkan skin types yang sesuai (nilai 1 hanya untuk skin_types yang dipilih)
    all_skin_types = ['sensitive', 'combination', 'dry', 'oily', 'normal']
    other_skin_types = [stype for stype in all_skin_types if stype not in skin_types]
    
    # Produk harus memiliki nilai 1 untuk semua skin_types yang dipilih dan 0 untuk lainnya
    skin_type_mask = (filtered_df[skin_types].sum(axis=1) == len(skin_types)) & (filtered_df[other_skin_types].sum(axis=1) == 0)
    filtered_df = filtered_df[skin_type_mask]
    
    # Hitung jumlah 1 di kolom skin problems yang sesuai dengan pilihan user
    total_skin_problems = filtered_df[['kulit_kusam', 'jerawat', 'bekas_jerawat', 'pori_pori_besar', 'flek_hitam',
                                       'garis_halus_dan_kerutan', 'komedo', 'warna_kulit_tidak_merata', 'kemerahan',
                                       'kulit_kendur']].sum(axis=1)
    skin_problems_match = filtered_df[skin_problems].sum(axis=1) / total_skin_problems
    
    # Hitung jumlah 1 di kolom notable effects yang sesuai dengan pilihan user
    total_notable_effects = filtered_df[['acne_free', 'soothing', 'brightening', 'moisturizing', 'hydrating',
                                         'pore_care', 'anti_aging', 'balancing', 'uv_protection', 'skin_barrier',
                                         'refreshing', 'oil_control', 'no_whitecast', 'black_spot']].sum(axis=1)
    notable_effects_match = filtered_df[notable_effects].sum(axis=1) / total_notable_effects
    
    # Gabungkan semua skor kesesuaian
    total_score = (skin_problems_match + notable_effects_match)/2
    
    # Urutkan produk berdasarkan total_score secara descending
    recommended_products = filtered_df.assign(total_score=total_score).sort_values(by='total_score', ascending=False)
    recommended_products = recommended_products[recommended_products['total_score'] > 0]
    
    return recommended_products[['product_name', 'brand', 'price', 'description', 'picture_src', 'total_score', 'rating', 'product_type']]

# Contoh penggunaan fungsi recommend_skincare dengan multiple input
skin_types = ['oily']
skin_problems = ['bekas_jerawat']
notable_effects = ['pore_care', 'brightening']

recommended_products = recommend_skincare('Toner', skin_types, skin_problems, notable_effects)
recommended_products

,product_name,brand,price,description,picture_src,total_score,rating,product_type
326,ERHA Acneact Clarifying Toner,ERHA,104500,Clarify Your Acne-Prone Skin with Acne Clarify...,https://www.beautyhaul.com/assets/uploads/prod...,0.750000,5,Toner
136,BHUMI Skin Ready Hydrating Booster,BHUMI,178500,Hydrating booster yang dirancang khusus untuk ...,https://www.beautyhaul.com/assets/uploads/prod...,0.666667,5,Toner
152,BHUMI Beneath Cloud Nine Essence Toner,BHUMI,126500,Lets calm your skin and fly to the Cloud Nine!...,https://www.beautyhaul.com/assets/uploads/prod...,0.666667,5,Toner
679,NARUKO Tea Tree Shine Control & Blemish Clear ...,NARUKO,300000,"Tea Tree Shine Control & Blemish Clear Toner, ...",https://www.beautyhaul.com/assets/uploads/prod...,0.625000,5,Toner
443,HAPLE Licorice Water,HAPLE,112500,"membantu mencerahkan, menyegarkan, menghidrasi...",https://www.beautyhaul.com/assets/uploads/prod...,0.625000,5,Toner
691,NATUR Miracle Calming Hydrating Toner,NATUR,139000,Natur Calming Hydrating Toner dengan non-irrit...,https://www.beautyhaul.com/assets/uploads/prod...,0.625000,5,Toner
675,NACIFIC Phyto Niacin Whitening Toner,NACIFIC,312000,NACIFIC (Natural Pacific) Phyto Niacin Whiteni...,https://www.beautyhaul.com/assets/uploads/prod...,0.625000,5,Toner
833,SCARLETT WHITENING Acne Essence Toner,SCARLETT,82500,Manfaat : Zinc Sebum meminimalkan kemerahan ya...,https://www.beautyhaul.com/assets/uploads/prod...,0.600000,5,Toner
3,ACWELL Aquaseal Soothing Tonic,ACWELL,290000,Pre-essence yang diformulasikan dengan ekstrak...,https://www.beautyhaul.com/assets/uploads/prod...,0.500000,5,Toner
537,IUNIK Vitamin Hyaluronic Acid Vitalizing Toner...,IUNIK,320000,IUNIK Vitamin Hyaluronic Acid Vitalizing Toner...,https://www.beautyhaul.com/assets/uploads/prod...,0.500000,5,Toner
